In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature_pressure import*

[03:33:51] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[03:33:52] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0.05
batch_size = 64

In [ ]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()

In [8]:
import pandas as pd
co2_database = pd.read_excel(os.path.join(root_path,'datasets/il properties/co2solubility_S.xlsx'),sheet_name='co2solubility')

In [10]:
train_IL_smiles = co2_database['IL SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_T =co2_database['normalized_T']
train_P = co2_database['normalized_P']
train_lngamma = co2_database['x_CO2']
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_IL_smiles,train_T,train_P,train_lngamma))
train_dataloader = get_train_dataloader(train_dataset, train_batch_size=train_batch_size)

In [ ]:
epochs, learning_rate = 160,0.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
max_len = 100
save_name = 'co2.best.params'
train(net, train_dataloader, batch_size, learning_rate, ctx, epochs,save_name=save_name)